In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# Load preprocessed dataset (Assuming it's already preprocessed and ready)
data = pd.read_csv("energy_consumption_preprocessed.csv")

# Feature selection (can customize based on data)
features = ['Temperature', 'Humidity', 'Wind Speed', 'Hour', 'Day', 'Weekday', 'Month', 'Holiday_1', 'Season_Summer', 'Season_Winter']
target = 'Consumption'

X = data[features]
y = data[target]

# Split the dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features using StandardScaler (important for linear models)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model initialization for Linear Regression and Random Forest Regressor
regressor_lr = LinearRegression()
regressor_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Training the Linear Regression model
print("Training the Linear Regression model...")
regressor_lr.fit(X_train_scaled, y_train)

# Hyperparameter tuning using GridSearchCV for Random Forest Regressor
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

print("Tuning Random Forest Regressor...")
grid_search_rf = GridSearchCV(estimator=regressor_rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search_rf.fit(X_train_scaled, y_train)

# Best parameters for Random Forest
print(f"Best parameters for Random Forest: {grid_search_rf.best_params_}")

# Model performance on the test set (Linear Regression)
y_pred_lr = regressor_lr.predict(X_test_scaled)
print(f"Linear Regression Model Performance:")
print(f"R^2 Score: {r2_score(y_test, y_pred_lr):.4f}")
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred_lr):.4f}")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_test, y_pred_lr)):.4f}")
print(f"Mean Absolute Error (MAE): {mean_absolute_error(y_test, y_pred_lr):.4f}")

# Predictions from Random Forest with the best parameters
y_pred_rf = grid_search_rf.predict(X_test_scaled)
print(f"Random Forest Model Performance:")
print(f"R^2 Score: {r2_score(y_test, y_pred_rf):.4f}")
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred_rf):.4f}")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_test, y_pred_rf)):.4f}")
print(f"Mean Absolute Error (MAE): {mean_absolute_error(y_test, y_pred_rf):.4f}")

# Comparing Linear Regression and Random Forest Model Predictions
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted LR': y_pred_lr,
    'Predicted RF': y_pred_rf
})

print(comparison_df.head())

# Saving the model predictions to CSV
comparison_df.to_csv("model_comparison_predictions.csv", index=False)

# Visualizing Actual vs Predicted Values for Linear Regression
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label='Actual Consumption', color='blue', linewidth=2)
plt.plot(y_pred_lr, label='Predicted Consumption (Linear Regression)', color='red', linestyle='dashed', linewidth=2)
plt.title('Actual vs Predicted Energy Consumption (Linear Regression)')
plt.xlabel('Data Points')
plt.ylabel('Consumption (kWh)')
plt.legend()
plt.grid(True)
plt.show()

# Visualizing Actual vs Predicted Values for Random Forest
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label='Actual Consumption', color='blue', linewidth=2)
plt.plot(y_pred_rf, label='Predicted Consumption (Random Forest)', color='green', linestyle='dashed', linewidth=2)
plt.title('Actual vs Predicted Energy Consumption (Random Forest)')
plt.xlabel('Data Points')
plt.ylabel('Consumption (kWh)')
plt.legend()
plt.grid(True)
plt.show()

# Saving the trained models using pickle
with open('linear_regression_model.pkl', 'wb') as lr_file:
    pickle.dump(regressor_lr, lr_file)

with open('random_forest_model.pkl', 'wb') as rf_file:
    pickle.dump(grid_search_rf.best_estimator_, rf_file)

with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

print("Models and scaler saved successfully!")

# Error Analysis (Residuals)
errors_lr = y_pred_lr - y_test.values
errors_rf = y_pred_rf - y_test.values

print(f"Linear Regression - Mean error: {np.mean(errors_lr)}")
print(f"Random Forest - Mean error: {np.mean(errors_rf)}")

# Residual analysis for Linear Regression
plt.figure(figsize=(12, 6))
sns.histplot(errors_lr, kde=True, color='purple', label="LR Residuals")
sns.histplot(errors_rf, kde=True, color='orange', label="RF Residuals")
plt.title('Residuals Distribution')
plt.xlabel('Prediction Error (kWh)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True)
plt.show()

# Residuals vs Predicted values plot for both models
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(y_pred_lr, errors_lr, color='purple', alpha=0.5)
plt.axhline(y=0, color='black', linestyle='--')
plt.title('Residuals vs Predicted Values (Linear Regression)')
plt.xlabel('Predicted Values (Linear Regression)')
plt.ylabel('Residuals')

plt.subplot(1, 2, 2)
plt.scatter(y_pred_rf, errors_rf, color='orange', alpha=0.5)
plt.axhline(y=0, color='black', linestyle='--')
plt.title('Residuals vs Predicted Values (Random Forest)')
plt.xlabel('Predicted Values (Random Forest)')
plt.ylabel('Residuals')

plt.tight_layout()
plt.show()

# Cross-Validation Score (for Linear Regression and Random Forest)
cv_score_lr = cross_val_score(regressor_lr, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
cv_score_rf = cross_val_score(grid_search_rf.best_estimator_, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')

print(f"Cross-Validation Score (Linear Regression): {np.mean(cv_score_lr):.4f}")
print(f"Cross-Validation Score (Random Forest): {np.mean(cv_score_rf):.4f}")

# Hyperparameter tuning for Linear Regression  - Ridge or Lasso
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_test_scaled)

print(f"Ridge Regression Model Performance:")
print(f"R^2 Score: {r2_score(y_test, y_pred_ridge):.4f}")
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred_ridge):.4f}")
print(f"Root Mean Squared Error (RMSE): {np.sqrt(mean_squared_error(y_test, y_pred_ridge)):.4f}")
print(f"Mean Absolute Error (MAE): {mean_absolute_error(y_test, y_pred_ridge):.4f}")
